In [131]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from lightgbm import LGBMClassifier
from sklearn.metrics import log_loss
from sklearn.metrics import accuracy_score,roc_auc_score
from xgboost import XGBClassifier

In [3]:
train_x = pd.read_csv("C:/Users\Administrator/Downloads/AV datathon/train.csv")
test = pd.read_csv("C:/Users\Administrator/Downloads/AV datathon/test.csv")

train_x.drop(['redemption_status'],axis =1,inplace = True)

train_x['train'] = "Yes"
test['train'] = "No"

train = pd.concat([train_x,test],axis =0,sort = False).reset_index()
train.drop('index',inplace = True,axis =1)

In [5]:
campaign_data = pd.read_csv("C:/Users\Administrator/Downloads/AV datathon/campaign_data.csv")
coupon_item_mapping = pd.read_csv("C:/Users\Administrator/Downloads/AV datathon/coupon_item_mapping.csv")
customer_demographics = pd.read_csv("C:/Users\Administrator/Downloads/AV datathon/customer_demographics.csv")
customer_transaction_data = pd.read_csv("C:/Users\Administrator/Downloads/AV datathon/customer_transaction_data.csv")
item_data =  pd.read_csv("C:/Users\Administrator/Downloads/AV datathon/item_data.csv")

In [7]:
customer_transaction_data['actual_price'] = customer_transaction_data["selling_price"] + (customer_transaction_data['other_discount'] * (-1)) + (customer_transaction_data['coupon_discount'] * (-1)) 
customer_transaction_data['diff_actual_selling_price'] = customer_transaction_data['actual_price']- customer_transaction_data['selling_price']
customer_transaction_data['ratio_actual_selling_price'] = customer_transaction_data['selling_price']/customer_transaction_data['actual_price'] 

coupon_used_transaction = customer_transaction_data[customer_transaction_data['coupon_discount'] != 0]

###customer_transaction_data aggregation
#customer_transaction_data.groupby('customer_id')['selling_price'].sum()

max_purchase = customer_transaction_data.groupby(['customer_id'],sort = True)['item_id','selling_price'].max()
max_purchase = max_purchase.rename(columns = {'item_id' : 'max_sell_price_item',"selling_price" : 'max_sell_price'})
max_actual_price_purchase = customer_transaction_data.groupby(['customer_id'],sort = True)['item_id','actual_price'].max()
max_actual_price_purchase = max_actual_price_purchase.rename(columns = {'item_id' : 'max_actual_price_item',"actual_price" : 'max_actual_price'})

max_actual_max_sell = pd.concat([max_actual_price_purchase,max_purchase],axis =1)

max_actual_max_sell = max_actual_max_sell.reset_index()
max_actual_max_sell.drop(['max_actual_price_item'],axis =1,inplace = True)

max_actual_max_sell['max_diff_actual_sell_price'] =  max_actual_max_sell['customer_id'].map(customer_transaction_data.groupby('customer_id')['diff_actual_selling_price'].max())
max_actual_max_sell['min_diff_actual_sell_price'] =  max_actual_max_sell['customer_id'].map(customer_transaction_data.groupby('customer_id')['diff_actual_selling_price'].min())
max_actual_max_sell['max_ratio_actual_selling_price'] =  max_actual_max_sell['customer_id'].map(customer_transaction_data.groupby('customer_id')['ratio_actual_selling_price'].max())
max_actual_max_sell['min_ratio_actual_selling_price'] =  max_actual_max_sell['customer_id'].map(customer_transaction_data.groupby('customer_id')['ratio_actual_selling_price'].min())

max_actual_max_sell['diff_max_actual_max_sell_price'] = max_actual_max_sell['max_actual_price'] - max_actual_max_sell['max_sell_price']
max_actual_max_sell['ratio_max_actual_max_sell_price'] = max_actual_max_sell['max_sell_price']/max_actual_max_sell['max_actual_price']

max_actual_max_sell['purchase_count'] = max_actual_max_sell['customer_id'].map(customer_transaction_data.groupby('customer_id')['selling_price'].count())
max_actual_max_sell['coupon_usage_count'] = max_actual_max_sell['customer_id'].map(coupon_used_transaction.groupby('customer_id')['coupon_discount'].count()).fillna(0)
max_actual_max_sell['ratio_purchase_count_discount_count'] = max_actual_max_sell['coupon_usage_count']/max_actual_max_sell['purchase_count']
max_actual_max_sell['coupon_usage_amt'] = max_actual_max_sell['customer_id'].map(coupon_used_transaction.groupby('customer_id')['coupon_discount'].sum()).fillna(0)
max_actual_max_sell['max_coupon_usage_amt'] = max_actual_max_sell['customer_id'].map(coupon_used_transaction.groupby('customer_id')['coupon_discount'].min()).fillna(0)

####in which transaction he used his first coupon
coupon_first_day = customer_transaction_data[["customer_id","date","coupon_discount"]].groupby(['customer_id','date'],sort = True).sum().reset_index().groupby(['customer_id']).head(1).reset_index()
coupon_first_day.drop(['index','date'],axis =1,inplace = True)

max_actual_max_sell = pd.merge(max_actual_max_sell,coupon_first_day,how = "left",on = "customer_id")

### Since the min_diff_actual_sell_price and max_ratio_actual_selling_price is same for all the customers we are removing that that variable
max_actual_max_sell.drop(['min_diff_actual_sell_price','max_ratio_actual_selling_price'],axis =1,inplace = True)

In [8]:
coupon_item_mapping['item_list'] =  coupon_item_mapping['coupon_id'].map(coupon_item_mapping.groupby(["coupon_id"])["item_id"].apply(list))

coupon_item_mapping.drop(['item_id'],axis =1,inplace = True)

coupon_item_mapping.drop_duplicates(subset = ['coupon_id'],inplace= True)

In [9]:
merged_data = pd.merge(train,campaign_data,how = "left",on = "campaign_id" )

merged_data = pd.merge(merged_data,coupon_item_mapping,how = "left",on = "coupon_id")

merged_data = pd.merge(merged_data,customer_demographics,how = "left" , on = "customer_id")

In [10]:
## transaction data
merged_data['total_purchase_price'] = merged_data['customer_id'].map(customer_transaction_data.groupby('customer_id')['selling_price'].sum())
merged_data = pd.merge(merged_data,max_purchase,how = "left", on = "customer_id")

In [11]:
merged_data['len_item_list'] = merged_data['item_list'].apply(lambda x : len(x))

merged_data.drop('item_list',axis =1,inplace = True)

In [13]:
merged_data.drop(['age_range','marital_status','rented','family_size','no_of_children','income_bracket'],axis =1,inplace = True)

In [14]:
merged_data = pd.merge(merged_data,max_actual_max_sell,how = "left", on = "customer_id",sort = True)

In [15]:
merged_data['start_date'] = pd.to_datetime(merged_data['start_date'],format = "%d/%m/%y")

merged_data['end_date'] = pd.to_datetime(merged_data['end_date'],format = "%d/%m/%y")

In [16]:
merged_data['campaign_days'] = merged_data['end_date'] - merged_data['start_date']

In [17]:
merged_data['start_date_month_end'] = merged_data['start_date'].dt.is_month_end
merged_data['end_date_month_end'] = merged_data['end_date'].dt.is_month_end
merged_data['start_date_year_end'] = merged_data['start_date'].dt.is_year_end
merged_data['start_date_quarter'] = merged_data['start_date'].dt.quarter

In [18]:
drop_columns = ['campaign_id','coupon_id','start_date','end_date']
merged_data.drop(drop_columns, axis = 1,inplace = True)

In [20]:
train_data = merged_data.loc[merged_data['train'] == 'Yes']
test_data = merged_data.loc[merged_data['train'] == 'No']

In [23]:
data = pd.read_csv("C:/Users\Administrator/Downloads/AV datathon/train.csv")
final_data = pd.merge(train_data,data[['id','redemption_status']], how = 'left',on = 'id')

In [24]:
final_data.drop(['train','customer_id','id','campaign_days'],axis =1,inplace = True)

In [25]:
le = LabelEncoder()
final_data['campaign_type'] = le.fit_transform(final_data['campaign_type'])

In [26]:
X = final_data.drop('redemption_status',axis =1)
Y = final_data['redemption_status']
### test train split
train_x,test_x,train_y,test_y = train_test_split(X,Y,random_state = 1000,test_size = 0.30)

In [27]:
lgb = LGBMClassifier(nthread=8,
                     n_estimators=400,
                     num_leaves=45,
                     learning_rate=0.03,
                     subsample=1.0,
                     subsample_freq=1,
                     colsample_bytree=0.3,
                     min_child_samples=10)

lgb.fit(train_x, train_y)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=0.3,
               importance_type='split', learning_rate=0.03, max_depth=-1,
               min_child_samples=10, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=400, n_jobs=-1, nthread=8, num_leaves=45,
               objective=None, random_state=None, reg_alpha=0.0, reg_lambda=0.0,
               silent=True, subsample=1.0, subsample_for_bin=200000,
               subsample_freq=1)

In [30]:
predict = lgb.predict(test_x)

In [560]:
roc_auc_score(test_y,predict)

0.5402829081819164

In [28]:
test_data.drop(['train','customer_id','id','campaign_days'],axis =1,inplace = True)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [36]:
le = LabelEncoder()
test_data['campaign_type'] = le.fit_transform(test_data['campaign_type'])

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [44]:
test_predict = pd.Series(lgb.predict(test_data))

In [32]:
submission = pd.read_csv("C:/Users/Administrator/Downloads/AV datathon/sample_submission.csv")

In [71]:
submission.head()

,id,redemption_status
0,3,0
1,4,0
2,5,0
3,8,0
4,10,0


In [100]:
submit_test_data = merged_data.loc[merged_data['train'] == 'No'].reset_index()
final = pd.concat([submit_test_data['id'],test_predict],axis =1)

In [101]:
final = final.rename(columns = {0 : "redemption_status" })

In [102]:
final = final.sort_values(by = 'id').reset_index()

In [103]:
final.drop('index',axis = 1,inplace = True)

In [96]:
final = final[['id','redemption_status']]

In [104]:
final.index = final['id']

In [108]:
final.drop('id',axis =1,inplace = True)

In [109]:
final.to_csv(r'C:/Users/Administrator/Downloads/AV datathon/final_output.csv')

In [111]:
pd.read_csv("C:/Users/Administrator/Downloads/AV datathon/final_output.csv")

,id,redemption_status
0,3,0
1,4,0
2,5,0
3,8,0
4,10,0
5,12,0
6,13,0
7,16,0
8,18,0
9,31,0


In [129]:
xgbmodel = XGBClassifier(max_depth = 12,n_estimators = 550,nthread=8,
                     num_leaves=45,
                     learning_rate=0.03,
                     subsample=1.0,
                     subsample_freq=1,
                     colsample_bytree=0.3,
                     min_child_samples=10)

xgbmodel.fit(train_x, train_y)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.3, gamma=0,
              learning_rate=0.03, max_delta_step=0, max_depth=12,
              min_child_samples=10, min_child_weight=1, missing=None,
              n_estimators=550, n_jobs=1, nthread=8, num_leaves=45,
              objective='binary:logistic', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, seed=None, silent=None,
              subsample=1.0, subsample_freq=1, verbosity=1)

In [130]:
predict = xgbmodel.predict(test_x)

roc_auc_score(test_y,predict)

0.5333973352458474

In [122]:
test_y.shape,predict.shape

((23511,), (23511,))

In [ ]:
annmodel = 